In [1]:
import os
home = os.getcwd()

In [2]:
# Example pipeline dictionary
pipeline = {
    "yolov3u_raw2crop.py": [1],
    "yolov8_raw2instance_withmask_bbcroppos_padding.py": [-1],
    "yolov8n_raw2crop.py": [1],
    "yolov8_raw2crop_bbpos.py": [1],
    "yolov3u_raw2crop_bbpos.py": [1],
    "yolov8_raw2instance_nobbcrop.py": [1],
    "yolov8_crop2instance.py": [2],
    "yolov8_crop2instance_padding.py": [2],
    "copy_script.py": [-2,4],
    "bg_remove_yolov8.py": [-4],
    "deeplab_bgrm_nomask.py": [4],
    "deeplab.py": [2],
    "sd_inpainting_r2i_dilate_withbg.py": [3],
    "sd_userguidance.py": [-3],
    "sd_inpainting_r2i_dilate.py": [3],
    "fine_tune.py": [5], 
    "fine_tune_bgremoved.py": [-5],
    "Super-resolution.py": [6,0], # 0 - This script will be skipped
    "Retouch.py": [7,0]
}

tab_names = ['Obj. detection', 'BG removal/ Instance seg.', 'Restoration', 'BG removal', 'Fine tuning','Image Editor', 'Restored Image']

def group_scripts(pipeline):
    grouped_scripts = {}  # Dictionary to store the grouped scripts

    for script, groups in pipeline.items():
        if 0 in groups:
            continue  # Skip scripts with '0' in the groups

        for group in groups:
            # Treat -1 as the same group as 1
            real_ = group
            group = abs(group)

            if group not in grouped_scripts:
                grouped_scripts[group] = []

            if real_ < 0:
                grouped_scripts[group].insert(0, script)
            else:
                grouped_scripts[group].append(script)

    return grouped_scripts
grouped_scripts = group_scripts(pipeline)

grouped_scripts = dict(sorted(grouped_scripts.items()))

In [3]:
import os
import shutil

def clear_location(location):
    # Check if the specified location exists
    if os.path.exists(location):
        try:
            # Use shutil.rmtree to remove all contents (directories and files)
            shutil.rmtree(location)
        except Exception as e:
            print(f"Error clearing location: {e}")

def create_directories(location, n):
    # Clear the location first
    clear_location(location)

    try:
        # Create n directories from 1 to n
        for i in range(1, n + 1):
            directory_name = os.path.join(location, str(i))
            os.makedirs(directory_name)
    except Exception as e:
        print(f"Error creating directories: {e}")

number_of_dir = len(grouped_scripts)+1
create_directories(home+"/Thesis/PipelineImages/",number_of_dir )

In [4]:
import os
import ipywidgets as widgets
from ipywidgets import Tab
from IPython.display import display, clear_output, Image, HTML, Javascript
import subprocess
import time


# Define the folder from which you want to allow image uploads
image_upload_folder = home+"/Thesis/Images/"  # Replace with your desired folder path

# Define the list of output directories for each tab
output_base_path = home+"/Thesis/PipelineImages/"
output_dirs = [os.path.join(output_base_path, str(i + 2)) for i in range(len(grouped_scripts))]

script_folder = home+"/Thesis/Codes/"
ImageEditorApp_path = home+"/Thesis/dragApp/app.py"


# Create a list of Python script options for each tab
python_scripts = [value for key, value in grouped_scripts.items()]

# Define the folder from which you want to allow image uploads
reg_image_upload_folder = home+"/Thesis/PipelineImages/" + str(len(python_scripts)) + '/'

# Create a list to store the execution status of each tab
tab_execution_status = [False] * len(python_scripts)


# Function to execute the next tab
def execute_next_tab(index):
    if not tab_execution_status[index]:
        # dropdowns[index].value = python_scripts[index][0]  # Select the first script in the dropdown     
        execute_py_file(index)  # Execute the selected script
        tab_execution_status[index] = True
        tabs.selected_index = index # Move to the next tab

# Function to execute all tabs sequentially
def execute_all_tabs():
    for i in range(len(python_scripts)-1):
        execute_next_tab(i)


# Create a button for executing all tabs
execute_all_button = widgets.Button(description="Automate")

# Create a button for resetting execution status
reset_execution_button = widgets.Button(description="Reset")

# Function to reset the execution status
def reset_execution_status(_):
    for i in range(len(python_scripts)):
        tab_execution_status[i] = False

# Set the click event to execute all tabs sequentially
execute_all_button.on_click(lambda btn: execute_all_tabs())

# Set the click event to reset the execution status
reset_execution_button.on_click(reset_execution_status)


# Function to execute the selected .py file when the "Process" button is pressed
def execute_py_file(change):
    # selected_tab = tabs.selected_index  # Get the index of the selected tab
    selected_tab = change
    selected_file = dropdowns[selected_tab].value
    selected_file_path = os.path.join(script_folder, selected_file)

    if selected_file == "None":
        with tab_outputs[selected_tab]:
            clear_output(wait=True)  # Clear the previous output within the tab
            print("No script selected.")
        return

    with tab_outputs[selected_tab]:
        clear_output(wait=True)  # Clear the previous output within the tab
        print("Running...")

    try:
        
        if selected_tab == 0: #For execution of the first tab; As it needs the input image too
            process = subprocess.Popen(["python", selected_file_path, bild_name, str(selected_tab + 1)],
                                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
        elif selected_tab == len(python_scripts) - 1: #Final tab from the python scripts
            process = subprocess.Popen(["python", selected_file_path, ft_name, text_input.value, str(selected_tab + 1)],
                                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        elif selected_tab == number_associated - 1: #The tab which has sd_userguidance.py; this number_associated is calculated in the bottom part of code
            
            if selected_file == 'sd_userguidance.py':
                process = subprocess.Popen(["python", selected_file_path, str(selected_tab + 1), str(slider_guidanceScale.value), str(slider_dilationScale.value)],
                               stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            else: 
                process = subprocess.Popen(["python", selected_file_path, str(selected_tab + 1)],
                                           stdout=subprocess.PIPE, stderr=subprocess.PIPE)    

        else:
            process = subprocess.Popen(["python", selected_file_path, str(selected_tab + 1)],
                                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        while process.poll() is None:
            time.sleep(0.5)

        with tab_outputs[selected_tab]:
            clear_output(wait=True)  # Clear the previous output within the tab
            out, err = process.communicate()
            result_labels[selected_tab].value = f"Executed: {selected_file}\n\n{out.decode('utf-8')}\n{err.decode('utf-8')}"

            # Display the processed image
            if selected_tab == len(python_scripts) - 1:
                display_processed_image(ft_name)

        # Display images from the output directory within the tab; The fine_tune.py file does not change the directory of the images, so displays from selected_tab - 1
        if selected_tab == len(python_scripts) - 1:
            display_images_from_output_dir(selected_tab - 1)
        else:
            display_images_from_output_dir(selected_tab)



        # If this is the last tab, update the image dropdown options
        if selected_tab == len(python_scripts) - 2:
            update_reg_image_dropdown_options()

    except subprocess.CalledProcessError:
        with tab_outputs[selected_tab]:
            clear_output(wait=True)  # Clear the previous output within the tab
            print(f"Error executing: {selected_file}")

    # If this is the last tab, clear the input prompt and update the image selection
    if selected_tab == len(python_scripts) - 1:
        text_input.value = ""  # Clear the input prompt
        reg_image_dropdown.value = "None"  # Reset image selection

# Function to display the processed image
def display_processed_image(processed_image_name):
    with output:
        output.clear_output()
        processed_image_path = os.path.join(reg_image_upload_folder, processed_image_name)
        processed_image = Image(filename=processed_image_path)
        display(processed_image)


# Function to handle image selection from the dropdown and display the selected image
def select_image(change):
    global bild_name  # Declare bild_name as a global variable
    selected_image = image_dropdown.value
    if selected_image != "None":
        bild_name = selected_image
        image_upload_label.value = f"Selected Image: {bild_name}"
        display_selected_image(selected_image)
    else:
        bild_name = None
        image_upload_label.value = "Selected Image: None"
        image_output.clear_output()  # Clear the output when no image is selected

# Function to display the selected image and replace any previous images
def display_selected_image(selected_image):
    with image_output:  # Use the 'output' widget to clear the output
        image_output.clear_output()
        image_path = os.path.join(image_upload_folder, selected_image)
        image = Image(filename=image_path)
        width, height = 200, 200  # Set a fixed width and height (adjust as needed)
        image.width = width
        image.height = height
        display(image)

# Function to handle image selection from the dropdown and display the selected image
def reg_select_image(change):
    global ft_name  # Declare fine tune image as a global variable
    reg_selected_image = reg_image_dropdown.value
    if reg_selected_image != "None":
        ft_name = reg_selected_image
        upload_label.value = f"Selected Image: {ft_name}"
        reg_display_selected_image(reg_selected_image)
    else:
        ft_name = None
        upload_label.value = ""
        output.clear_output()  # Clear the output when no image is selected

# Function to display the selected image and replace any previous images
def reg_display_selected_image(reg_selected_image):
    with output:  # Use the 'output' widget to clear the output
        output.clear_output()
        image_path = os.path.join(reg_image_upload_folder, reg_selected_image)
        image = Image(filename=image_path)
        display(image)

# Function to update the image dropdown options in the last tab
def update_reg_image_dropdown_options():
    # Get the list of image files in the regenerated folder
    reg_image_files = [f for f in os.listdir(reg_image_upload_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    reg_image_dropdown.options = ['None'] + reg_image_files

# Function to generate and display a link
def display_link(b):
    with link_output:
        # Display a link to a website
        link_html = '<a href="http://127.0.0.1:5000" target="_blank">Go to Image Editor</a>'
        display(HTML(link_html))

# Function to display the CANVAS OUTPUT IMAGE
def display_canvas(b):
    with display_output:  # Use the 'output' widget to clear the output
        display_output.clear_output()
        canvas_image_path = os.path.join(output_base_path, str(len(python_scripts)+1), 'canvas_image.jpg')
        canvasimage = Image(filename=canvas_image_path)
        width, height = 200, 200  # Set a fixed width and height (adjust as needed)
        canvasimage.width = width
        canvasimage.height = height
        display(canvasimage)


def execute_app_py(b):
    process = subprocess.Popen(["python", ImageEditorApp_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # Wait for the process to complete
    process.wait()

# Create a list to store Image widgets for images in each tab
output_images = [[] for _ in range(len(python_scripts))]

# Function to display images from the output directory within the tab
def display_images_from_output_dir(selected_tab):
    output_dir = output_dirs[selected_tab]

    # Get the list of image files in the output directory
    image_files = [f for f in os.listdir(output_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

    if image_files:
        images = []  # Create a list to store Image widgets for images
        for image_file in image_files:
            image_path = os.path.join(output_dir, image_file)
            image = Image(filename=image_path)

            # Resize the image to 64x64 pixels
            image.value = open(image_path, "rb").read()
            image.format = 'image/png'
            image.width = 64
            image.height = 64
            
            images.append(image)  # Add each Image widget to the list
        output_images[selected_tab] = images  # Store the images for this tab
    else:
        output_images[selected_tab] = [widgets.Label(value="No images found in the folder")]

    # Update the output for the current tab
    with outputs[selected_tab]:
        clear_output()
    
        image_outputs = [widgets.Output() for _ in range(len(output_images[selected_tab]))]
    
        # Create an empty list to hold the HBoxes for each row
        rows = []
    
        # Display each image in its respective Output widget
        for i, image in enumerate(output_images[selected_tab]):
            with image_outputs[i]:
                display(image)
            # Add the current Output widget to the current row
            if i % 4 == 0:
                rows.append(widgets.HBox(image_outputs[i:i+4]))
    
        # Create a VBox to display rows of images
        vbox_output = widgets.VBox(rows)
    
        display(vbox_output)  # Display the VBox with rows of images


# Get the list of image files in the specified folder
image_files = [f for f in os.listdir(image_upload_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

# Create a list to store image dropdowns
dropdowns = []
tab_outputs = [widgets.Output() for _ in range(len(python_scripts))]

# Create and configure a single dropdown menu for image selection
image_dropdown = widgets.Dropdown(options=['None'] + image_files, description='Select Image:')
image_dropdown.observe(select_image, names='value')
image_output = widgets.Output()
image_upload_label = widgets.Label(value="")
# Create a VBox for the image dropdown and labels
image_box = widgets.VBox([image_dropdown, image_upload_label, image_output])


# Create a list to store widgets for each tab
tabs = widgets.Tab()
tab_contents = []

#Find the Tab index which has the Stable Diffusion code which supports user guidance 
script_to_find = "sd_userguidance.py"  # Replace with the script name you want to find
number_associated = next((number for number, scripts in grouped_scripts.items() if script_to_find in scripts), None)

# Create and configure widgets for each tab
for i, script_options in enumerate(python_scripts):
    if i == len(python_scripts) - 1:
        dropdown = widgets.Dropdown(options=script_options, description=f'Select Python Script {i + 1}:')
        dropdowns.append(dropdown)
        text_input = widgets.Text(value='', placeholder='Enter prompt here', description='Prompt Input:', disabled=False)

        # Get the list of image files in the regenerated folder
        reg_image_files = [f for f in os.listdir(reg_image_upload_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]
        # Create and configure a single dropdown menu for image selection
        reg_image_dropdown = widgets.Dropdown(options=['None'] + reg_image_files, description='Select Image:')
        reg_image_dropdown.observe(reg_select_image, names='value')

        # Remove the 'observe' line that listens for changes in the dropdown
        process_button = widgets.Button(description="Process")
        process_button.on_click(lambda btn: execute_py_file(tabs.selected_index))  # Set the button click event to execute the script
        result_label = widgets.Output()
        output = widgets.Output()
        tab_content = widgets.VBox([dropdown, reg_image_dropdown, text_input, process_button, result_label, output])
    elif i == number_associated-1:
        dropdown = widgets.Dropdown(options=script_options, description=f'Select Python Script {i + 1}:')
        dropdowns.append(dropdown)
        
        # Create a Guidance Scale slider widget
        slider_guidanceScale = widgets.FloatSlider(
            value=20.0,    # Initial value
            min=1.0,      # Minimum value
            max=20.0,      # Maximum value
            step=0.1,    # Step size
            description='Guidance Scale:',
        )
        # Create a slider widget
        slider_dilationScale = widgets.IntSlider(
            value=3,    # Initial value
            min=0,      # Minimum value
            max=10,      # Maximum value
            step=1,    # Step size
            description='Dilation Iterations:',
        )
        # Remove the 'observe' line that listens for changes in the dropdown
        process_button = widgets.Button(description="Process")
        process_button.on_click(lambda btn: execute_py_file(tabs.selected_index))  # Set the button click event to execute the script
        result_label = widgets.Output()
        upload_label = widgets.Label(value="")
        output = widgets.Output()
        tab_content = widgets.VBox([dropdown, slider_guidanceScale, slider_dilationScale, process_button, result_label, upload_label, output])
    else:
        dropdown = widgets.Dropdown(options=script_options, description=f'Select Python Script {i + 1}:')
        dropdowns.append(dropdown)
        # Remove the 'observe' line that listens for changes in the dropdown
        process_button = widgets.Button(description="Process")
        process_button.on_click(lambda btn: execute_py_file(tabs.selected_index))  # Set the button click event to execute the script
        result_label = widgets.Output()
        upload_label = widgets.Label(value="")
        output = widgets.Output()
        tab_content = widgets.VBox([dropdown, process_button, result_label, upload_label, output])
    tab_contents.append(tab_content)

#CREATE A TAB FOR IMAGE EDITOR APP
# Create a button and output for running the dragApp and displaying the link
editor_button = widgets.Button(description="Process")
link_output = widgets.Output()
dragApp_content =  widgets.VBox([editor_button, link_output])
#Add dragApp contents to Tab
tab_contents.append(dragApp_content)
# Attach the function to the button's click event
editor_button.on_click(display_link)
editor_button.on_click(execute_app_py)

#CREATE A TAB FOR FINAL CANVAS DISPLAY
# Create a button and output for running the dragApp and displaying the link
display_button = widgets.Button(description="Display Output")
display_output = widgets.Output()
canvas_content =  widgets.VBox([display_button, display_output])
#Add dragApp contents to Tab
tab_contents.append(canvas_content)
# Attach the function to the button's click event
display_button.on_click(display_canvas)


# Set the tab titles and contents
tabs.children = tab_contents
for i in range(len(tab_names)):
    tabs.set_title(i, tab_names[i])

# Display all widgets in a VBox
upload_labels = [tab.children[3] for tab in tabs.children[:len(python_scripts)]]
outputs = [tab.children[4] for tab in tabs.children[:len(python_scripts)]]
result_labels = [tab.children[2] for tab in tabs.children[:len(python_scripts)]]

# Create a HBox to display the image selection and the tabs side by side
tab_box = widgets.HBox([image_box, tabs])


# Display the "Execute all tabs" button and the "Reset Execution" button along with other widgets
display(widgets.VBox([execute_all_button, reset_execution_button, tab_box]))



In [11]:
import subprocess
# extrafiles = ['hartmanns_distorted.png']

# # for file in extra_files:
# result = %run "/beegfs/.global0/ws/tova651c-taurusbackup/Thesis/Codes/yolov8_raw2instance_withmask_bbcroppos_padding_autocode.py" {filename} 1
# result = %run "/beegfs/.global0/ws/tova651c-taurusbackup/Thesis/Codes/copy_script.py" 2
# result3 = %run "/beegfs/.global0/ws/tova651c-taurusbackup/Thesis/Codes/sd_userguidance.py" 3 20 2
# result4 = %run "/beegfs/.global0/ws/tova651c-taurusbackup/Thesis/Codes/bg_remove_yolov8.py" 4
result5 = %run "/beegfs/.global0/ws/tova651c-taurusbackup/Thesis/Codes/autoarrange.py"


Image file not found: http://127.0.0.1:5000/image/hartmanns_distorted_segemented_image_[zebra]_0.png. Skipping...
Image file not found: http://127.0.0.1:5000/image/hartmanns_distorted_segemented_image_[zebra]_1.png. Skipping...
Image file not found: http://127.0.0.1:5000/image/hartmanns_distorted_background.png. Skipping...


TypeError: expected str, bytes or os.PathLike object, not NoneType